In [2]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

import joypy

import pyBigWig

# Metadata G25

In [2]:
path = '/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/'

# gencode 25
metadata_pc_g25 = pd.read_csv(path+'tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()] 

metadata_pc_g25[0:2]

,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id,tr_id1
0,ENST00000335137.3,OR4F5,OR4F5-001,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,NaN,ATG,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,0,918,0,0,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...,ENST00000335137
1,ENST00000624431.1,FO538757.2,FO538757.2-201,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTA...,AAG,ATGAGTGACAGCATCAACTTCTCTCACAACCTAGGCCAGCTCCTGT...,ATG,TAG,316,718,0,316,ENST00000624431.1|ENSG00000279928.1|-|-|FO5387...,ENST00000624431


#### get only complete CDS

In [3]:
tmp = metadata_pc_g25[(metadata_pc_g25['utr5_end'].notna())]
tmp2 = tmp[tmp['utr5_end'] > 0]

tmp2['tr_len'] = tmp2['transcript_seq'].str.len()
tmp3 = tmp2[tmp2['tr_len'] > tmp2['cds_stop_pos']]

print (metadata_pc_g25.shape[0], tmp2.shape[0], tmp3.shape[0])

94359 78873 60109


/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
d = dict(tuple(tmp3[['tr_id', 'gene', 'cds_start_codon', 'utr5_end']].groupby(by='gene')))

In [5]:
for gene, df in d.items():
    starts = set()
    for el in df.to_numpy():
        tr_id = el[0]
        cds_start_codon = el[2]
        if cds_start_codon in ['CTG', 'TTG', 'GTG', 'ACG', 'AGG', 'AAG', 'ATT', 'ATA', 'ATC']:
            print (gene, tr_id, cds_start_codon)
        starts.add(cds_start_codon)
    
  

AL008723.1 ENST00000623133.1 CTG
ARAP1 ENST00000455638.6 ATA
ATP13A3 ENST00000619199.4 ATT
BAG1 ENST00000472232.7 CTG
C16orf82 ENST00000418886.1 CTG
C1orf234 ENST00000622840.1 AGG
CACNG8 ENST00000270458.2 CTG
CASK ENST00000378179.7 CTG
DDX17 ENST00000396821.7 CTG
DDX17 ENST00000403230.1 CTG
EBF1 ENST00000622875.4 AGG
EIF4G2 ENST00000526148.5 GTG
EIF4G2 ENST00000525681.5 GTG
EIF4G2 ENST00000339995.9 GTG
EIF4G2 ENST00000396525.6 GTG
EIF4G2 ENST00000531647.5 GTG
EIF4G2 ENST00000528562.5 GTG
FGF2 ENST00000614010.4 CTG
FGF2 ENST00000264498.7 CTG
FNDC5 ENST00000373471.7 ATA
GTF3A ENST00000381140.8 CTG
GTF3A ENST00000435781.5 CTG
GTF3A ENST00000419181.5 CTG
GTF3A ENST00000438571.5 CTG
HCK ENST00000534862.5 CTG
HCK ENST00000486475.5 CTG
HCK ENST00000262651.3 CTG
HCK ENST00000375852.2 CTG
HMHB1 ENST00000289448.2 CTG
KCNIP1 ENST00000390656.8 AAG
MDFIC ENST00000257724.7 GTG
MDFIC ENST00000614186.4 GTG
MDGA2 ENST00000426342.5 TTG
MRVI1 ENST00000527509.6 CTG
MYC ENST00000613283.1 CTG
MYC ENST000006

In [6]:
d = {'EIF4G2': [493, 1111, 'ENST00000339995'],  # nonAUG, dAUG1, tr_id 
     'BAG1': [88, 301, 'ENST00000472232'], 
     'R3HCC1': [269, 830, 'ENST00000265806'], 
     'GTF3A': [125, 900, 'ENST00000381140'], 
     'DDX17': [76, 313, 'ENST00000403230'], 
     'RNF187': [429, 756, 'ENST00000305943'],
     'HCK': [229, 292, 'ENST00000375862'], 
     'NELFB': [194, 338, 'ENST00000343053'],
     
    }

AL008723.1 ENST00000623133.1 CTG - not in G41 anymore 

ARAP1 ENST00000455638.6 ATA - removed 

ATP13A3 ENST00000619199.4 ATT - not in G41 anymore 

BAG1 ENST00000472232.7 CTG - OK

C16orf82 ENST00000418886.1 CTG - testis-specific, has goos MS spectra  

C1orf234 ENST00000622840.1 AGG - not in G41 anymore

CACNG8 ENST00000270458.2 CTG

CASK ENST00000378179.7 CTG

DDX17 ENST00000396821.7 CTG  OK
DDX17 ENST00000403230.1 CTG  OK 

EBF1 ENST00000622875.4 AGG - will be removed 

EIF4G2 ENST00000526148.5 GTG
EIF4G2 ENST00000525681.5 GTG
EIF4G2 ENST00000339995.9 GTG
EIF4G2 ENST00000396525.6 GTG
EIF4G2 ENST00000531647.5 GTG
EIF4G2 ENST00000528562.5 GTG

FGF2 ENST00000614010.4 CTG
FGF2 ENST00000264498.7 CTG

FNDC5 ENST00000373471.7 ATA - ? both ATG and ATA. no support for exon with ATA. Upstream ATG is annotated in human and mouse  

GTF3A ENST00000381140.8 CTG
GTF3A ENST00000435781.5 CTG
GTF3A ENST00000419181.5 CTG
GTF3A ENST00000438571.5 CTG

HCK ENST00000534862.5 CTG - a lot of transcripts, including in-frame cdsATGs, CTG seems genuine 
HCK ENST00000486475.5 CTG
HCK ENST00000262651.3 CTG
HCK ENST00000375852.2 CTG

HMHB1 ENST00000289448.2 CTG - ? not conserved , rna-seq is in intron etc. weird gene in MANE, abberant one, kept 

KCNIP1 ENST00000390656.8 AAG - rna-seq/ribo-seq in intron... there is tr_id with dATG as a start it's removed now

MDFIC ENST00000257724.7 GTG gtg is not conserved ribo-seq is not convincing it's gone in g41
MDFIC ENST00000614186.4 GTG

---------------

MDGA2 ENST00000426342.5 TTG more like internal one, no ribo-seq support 

MRVI1 ENST00000527509.6 CTG, internal CTG, no ribo-seq 

MYC ENST00000613283.1 CTG
MYC ENST00000621592.5 CTG
MYC ENST00000524013.1 CTG

NELFB ENST00000343053.4 CTG exslucive nonAUG/predominant? sink ORF 

NPTXR ENST00000333039.2 CTG exslucive nonAUG/predominant? dAUG is not even annotated 

NPW ENST00000329610.4 CTG - predominant CTG. but there is another weird transcript with upstream exon no support 

NR1I2 ENST00000393716.6 CTG  - no ribo-seq for this CTG, CTG is ~ok in mammals as well as dAUG. why was it annotated in a first place  
NR1I2 ENST00000466380.5 CTG

OAZ3 ENST00000479764.6 CTG
OAZ3 ENST00000400999.6 CTG

PEG10 ENST00000615790.4 CTG
PEG10 ENST00000612941.1 CTG

PIGX ENST00000392391.7 CTG exclusive no sink there is dAUG isoform wit shorter 5'leader 

POU5F1 ENST00000620031.4 CTG no ribo-seq more like internal one 
POU5F1 ENST00000606567.5 CTG

PRAMEF18 ENST00000624297.1 ATC removed in G41 
 
PRPS1L1 ENST00000506618.3 ACG - one exon gene with mappability issue. in trips looks good with amb reads like predominantly; it's ACG in human but ATG in other species. pc signal is fine 

PRR16 ENST00000446965.2 GTG, thre is dATG. could be exclusive, sink ORF 

R3HCC1 ENST00000522012.5 CTG
R3HCC1 ENST00000625275.2 CTG
R3HCC1 ENST00000265806.10 CTG

RNF187 ENST00000305943.7 CTG excl ctg 

SCNN1D ENST00000616713.4 ATC not a lot of ribo-seq, is not in G41 anymore 

SLC22A17 ENST00000397260.7 CTG exlusive CTG? G41 has only CTG. G25 and refseq has dATG 
SLC22A17 ENST00000474057.5 CTG
SLC22A17 ENST00000636431.1 CTG

SORCS1 ENST00000612154.4 ATA  ATA right in front of ATG. there is upstream ATG. no ribo-seq.  
SORCS1 ENST00000622431.4 ATA

SP3 ENST00000418194.6 ATA could be predominant in this articular trascrniot , there are CUGCUG 

STIM2 ENST00000467087.5 TTG ok but there was upstream ATG 
STIM2 ENST00000467011.5 TTG
STIM2 ENST00000465503.5 TTG

STK11IP ENST00000295641.14 AGG it's not in G41. but in trips-viz it looks suprisingly okay as a start  

SWI5 ENST00000608796.5 CTG looks like exclusive nonAUG
SWI5 ENST00000419867.6 CTG
SWI5 ENST00000418976.1 CTG

TEAD1 ENST00000527636.5 ATT
TEAD1 ENST00000527575.5 ATT

TEAD3 ENST00000338863.11 ATA

TEAD4 ENST00000358409.6 TTG
TEAD4 ENST00000359864.6 TTG
TEAD4 ENST00000443986.2 TTG

TGM5 ENST00000622115.1 CTG weird short exon; does not exist in G41 

TRPV6 ENST00000359396.7 ACG

TXNRD3 ENST00000524230.6 CTG exclusive 
TXNRD3 ENST00000523403.2 CTG

VEGFA ENST00000611736.4 CTG
VEGFA ENST00000621747.4 CTG
VEGFA ENST00000615393.4 CTG
VEGFA ENST00000617771.4 CTG
VEGFA ENST00000372067.7 CTG

WT1 ENST00000379077.7 CTG there are CTG and ATG peaks 
WT1 ENST00000332351.7 CTG
WT1 ENST00000448076.7 CTG

ZNF804A ENST00000613975.1 AGG, a lot of rna-seq data in introns; no such isoform in G41

# Transcripts/genes with predominant/exclusive nonAUG from annotation & ribo-seq. GENCODE25 

In [23]:
d = [
    ['EIF4G2', 493, 1111, 'ENST00000339995', 'sink_ORF'],  # nonAUG, dAUG1, tr_id 
     ['BAG1', 88, 301, 'ENST00000472232', 'sink_ORF'], 
     ['R3HCC1', 269, 830, 'ENST00000265806', 'sink_ORF'], 
     ['GTF3A', 125, 900, 'ENST00000381140', 'sink_ORF'], 
     ['DDX17', 76, 313, 'ENST00000403230', 'sink_ORF'], 
     ['RNF187', 429, 756, 'ENST00000305943', 'sink_ORF'],
     ['HCK', 229, 292, 'ENST00000375862', 'sink_ORF'], 
     ['NELFB', 194, 338, 'ENST00000343053', 'sink_ORF'], 
     ['NPTXR', 125, 158, 'ENST00000333039', 'sink_ORF'], 
     ['NPW', 383, 539, 'ENST00000329610', 'sink_ORF'], 
     ['PIGX', 176, 299, 'ENST00000392391', 'sink_ORF'],
     ['PRPS1L1', 82, 282,'ENST00000506618', 'sink_ORF'],
     ['PRR16', 181, 232, 'ENST00000446965', 'sink_ORF'],
     ['SLC22A17', 120, 453, 'ENST00000397260', 'sink_ORF'], 
     ['SP3', 385, 1036, 'ENST00000418194', 'sink_ORF'],
     ['SWI5', 120, 438, 'ENST00000608796', 'sink_ORF'],
     ['TXNRD3', 166, 712, 'ENST00000524230', 'sink_ORF'],
     ['CCDC8', ...]   # annotated in GENCODE41, we take correcponsing GENCODE25 transcript and move CDS annotation 

]

In [27]:
df = pd.DataFrame(d, columns = ['gene', 'nonAUG_start', 'AUG1_start', 'tr_id1', 'mechanism']).merge(metadata_pc_g25[['tr_id', 'tr_id1']], on='tr_id1', how='inner')

df['dist_between_AUG_nonAUG_codons'] = (df['AUG1_start'] - df['nonAUG_start']) // 3

df.sort_values(by='dist_between_AUG_nonAUG_codons')

,gene,nonAUG_start,AUG1_start,tr_id1,mechanism,tr_id,dist_between_AUG_nonAUG_codons
8,NPTXR,125,158,ENST00000333039,sink_ORF,ENST00000333039.2,11
12,PRR16,181,232,ENST00000446965,sink_ORF,ENST00000446965.2,17
6,HCK,229,292,ENST00000375862,sink_ORF,ENST00000375862.6,21
10,PIGX,176,299,ENST00000392391,sink_ORF,ENST00000392391.7,41
7,NELFB,194,338,ENST00000343053,sink_ORF,ENST00000343053.4,48
9,NPW,383,539,ENST00000329610,sink_ORF,ENST00000329610.4,52
11,PRPS1L1,82,282,ENST00000506618,sink_ORF,ENST00000506618.3,66
1,BAG1,88,301,ENST00000472232,sink_ORF,ENST00000472232.7,71
4,DDX17,76,313,ENST00000403230,sink_ORF,ENST00000403230.1,79
15,SWI5,120,438,ENST00000608796,sink_ORF,ENST00000608796.5,106


In [ ]:
create profiles for all of them 
alignments: avoidaance of in-frame AUG? 
sink ORFs 

# Metadata G41

In [4]:
def extrac_metadata(filepath, pc=True):
    if pc==True:
        troubles = []
        li = []
        with open(filepath, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                record_id = str(record.id)
                tr_id = record_id.split('|')[0]
                seq = str(record.seq)
                
                gene = record_id.split('|')[5]
                gene_tr = record_id.split('|')[4]
                
                # CDS extraction
                m_cds = re.finditer(r'(CDS:)[0-9]*-[0-9]*', record_id)
                cds = [x.group(0) for x in m_cds][0] #UTR3:955-1054
                cds_start = int(cds.split(':')[1].split('-')[0]) - 1
                cds_end = int(cds.split(':')[1].split('-')[1]) 
                cds_seq = seq[cds_start:cds_end]     
                cds_start_codon = cds_seq[0:3]
                cds_stop_codon = cds_seq[-3:]
        
                
                # UTR5 extraction
                # some transcript do not have UTR5 annotated 
                # but we still want to save such trascripts
                m_utr5 = re.finditer(r'(UTR5:)[0-9]*-[0-9]*', record_id)
                t = [x.group(0) for x in m_utr5]
                
                
                if t:
                    utr5 = t[0] #UTR5:1-1054
                    utr5_start = int(utr5.split(':')[1].split('-')[0]) - 1
                    utr5_end = int(utr5.split(':')[1].split('-')[1]) 
        
                    if utr5_start == utr5_end: #1=1
                        print ('utr5_start == utr5_end', record_id)
                        utr5_start_seq = seq[utr5_start-1:(utr5_end)]
                    else: 
                        utr5_start_seq = seq[utr5_start:(utr5_end)]
                        
                    li.append([tr_id, gene, gene_tr, seq, utr5_start_seq, 
                   utr5_start_seq[-3:], cds_seq,
                  cds_start_codon, cds_stop_codon, cds_start, cds_end,
                  utr5_start, utr5_end, record_id])
                    
                    
                else:
                    utr5_start = 0
                    utr5_end = 0
                    utr5_start_seq = None
                    cds_start = 0
                    
                    li.append([tr_id, gene, gene_tr, seq, utr5_start_seq, 
                   cds_seq[:3], cds_seq,
                  cds_start_codon, cds_stop_codon, cds_start, cds_end,
                  utr5_start, utr5_end, record_id])
        
        metadata_pc_g25 = pd.DataFrame(li, columns = ['tr_id', 'gene' ,'gene_tr', 
                                 'transcript_seq', '5UTR_start_seq', 
                                 'CDS_start', 'cds_seq', 'cds_start_codon', 
                                 'cds_stop_codon', 'cds_start_pos',  'cds_stop_pos',
                                 'utr5_start', 'utr5_end', 'record_id'])
        return (metadata_pc_g25)

In [6]:
metadata_pc_g41 = extrac_metadata(filepath="data/gencode.v41.pc_transcripts.fa", pc=True)
print ('# trs with 5UTR:', metadata_pc_g41[metadata_pc_g41['5UTR_start_seq'].notna()].shape[0], '; # all trs', metadata_pc_g41.shape[0])
metadata_pc_g41.to_csv('tmp_res/metadata_pc_g41.txt', sep='\t', index=False)

metadata_pc_g41[0:2]

# trs with 5UTR: 93459 ; # all trs 110224


,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
0,ENST00000641515.2,OR4F5,OR4F5-201,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,GTT,ATGAAGAAGGTAACTGCAGAGGCTATTTCCTGGAATGAATCAACGA...,ATG,TAG,60,1041,0,60,ENST00000641515.2|ENSG00000186092.7|OTTHUMG000...
1,ENST00000426406.4,OR4F29,OR4F29-201,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,None,ATG,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,ATG,TAA,0,939,0,0,ENST00000426406.4|ENSG00000284733.2|OTTHUMG000...


In [7]:
tmp = metadata_pc_g41[(metadata_pc_g41['utr5_end'].notna())]
tmp2 = tmp[tmp['utr5_end'] > 0]

tmp2['tr_len'] = tmp2['transcript_seq'].str.len()
tmp3 = tmp2[tmp2['tr_len'] > tmp2['cds_stop_pos']]

d = dict(tuple(tmp3[['tr_id', 'gene', 'cds_start_codon', 'utr5_end']].groupby(by='gene')))

for gene, df in d.items():
    starts = set()
    for el in df.to_numpy():
        tr_id = el[0]
        cds_start_codon = el[2]
        if cds_start_codon in ['CTG', 'TTG', 'GTG', 'ACG', 'AGG', 'AAG', 'ATT', 'ATA', 'ATC']:
            print (gene, tr_id, cds_start_codon)
        starts.add(cds_start_codon)

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


ARAP1 ENST00000455638.6 ATA
BAG1 ENST00000634734.3 CTG
C16orf82 ENST00000505035.3 CTG
CACNG8 ENST00000270458.4 CTG
CCDC8 ENST00000697726.1 CTG
DDX17 ENST00000403230.3 CTG
DDX17 ENST00000396821.8 CTG
EBF1 ENST00000622875.4 AGG
EFCAB14 ENST00000672422.2 CTG
EIF4G2 ENST00000526148.6 GTG
EIF4G2 ENST00000525681.6 GTG
EIF4G2 ENST00000339995.11 GTG
EIF4G2 ENST00000396525.7 GTG
EIF4G2 ENST00000531647.6 GTG
EIF4G2 ENST00000528562.6 GTG
FGF2 ENST00000264498.8 CTG
FNDC5 ENST00000373471.9 ATA
FNDC5 ENST00000649537.2 ATA
GTF3A ENST00000381140.10 CTG
GTF3A ENST00000435781.5 CTG
GTF3A ENST00000419181.5 CTG
GTF3A ENST00000438571.5 CTG
HCK ENST00000486475.6 CTG
HCK ENST00000262651.4 CTG
HCK ENST00000375852.5 CTG
HMHB1 ENST00000289448.4 CTG
IRAG1 ENST00000527509.7 CTG
KCNIP1 ENST00000390656.8 AAG
KCTD11 ENST00000333751.8 ATT
MDGA2 ENST00000426342.7 TTG
MORN2 ENST00000644631.4 CTG
MRTFA ENST00000651595.2 GTG
MRTFA ENST00000355630.10 GTG
MRTFA ENST00000402042.7 GTG
MYC ENST00000524013.2 CTG
MYC ENST000006

In [ ]:
ARAP1 ENST00000455638.6 ATA will be removed

BAG1 ENST00000634734.3 CTG

C16orf82 ENST00000505035.3 CTG - testis-specific, has goos MS spectra

CACNG8 ENST00000270458.4 CTG

CCDC8 ENST00000697726.1 CTG

DDX17 ENST00000403230.3 CTG
DDX17 ENST00000396821.8 CTG

EBF1 ENST00000622875.4 AGG will be removed

EFCAB14 ENST00000672422.2 CTG it's extension according to ribo-seq and aln but not top ranked 

EIF4G2 ENST00000526148.6 GTG
EIF4G2 ENST00000525681.6 GTG
EIF4G2 ENST00000339995.11 GTG
EIF4G2 ENST00000396525.7 GTG
EIF4G2 ENST00000531647.6 GTG
EIF4G2 ENST00000528562.6 GTG

FGF2 ENST00000264498.8 CTG

FNDC5 ENST00000373471.9 ATA no ribo-seq for ATA. In human it's ATA, in other species its ATG   
FNDC5 ENST00000649537.2 ATA

GTF3A ENST00000381140.10 CTG
GTF3A ENST00000435781.5 CTG
GTF3A ENST00000419181.5 CTG
GTF3A ENST00000438571.5 CTG

HCK ENST00000486475.6 CTG
HCK ENST00000262651.4 CTG
HCK ENST00000375852.5 CTG

HMHB1 ENST00000289448.4 CTG it's kept in MANE, it's aberrant gene 

IRAG1 ENST00000527509.7 CTG it has sinkORFs, dAUG is far away but no ribo-seq support

KCNIP1 ENST00000390656.8 

KCTD11 ENST00000333751.8 ATT, ATT in aln looks ok though upstream GUG in ribo-seq is better 
 
MDGA2 ENST00000426342.7 TTG esnembl only can be removed 

MORN2 ENST00000644631.4 CTG ribo-seq seems okay 

MRTFA ENST00000651595.2 GTG looks ok according to ribo-seq. It was not in RiboSET since there was ovlp with coding exons 
MRTFA ENST00000355630.10 GTG
MRTFA ENST00000402042.7 GTG

MYC ENST00000524013.2 CTG
MYC ENST00000621592.8 CTG

NELFB ENST00000343053.6 CTG

NFKBID ENST00000641389.2 CTG

NPTXR ENST00000333039.4 CTG

NPW ENST00000566435.4 CTG

NR1I2 ENST00000466380.6 CTG
NR1I2 ENST00000393716.8 CTG

OAZ3 ENST00000479764.7 CTG
OAZ3 ENST00000400999.7 CTG

OR8G3P ENST00000641572.1 GTG CHECK in OTTER, no ribo-seq support, only GTG isoform 
OR8G3P ENST00000641848.1 GTG

PANK2 ENST00000643504.2 CTG a lot of isoforms with atg, ctg and alt 5'ends 
PANK2 ENST00000610179.7 CTG
PANK2 ENST00000336066.8 CTG

PEG10 ENST00000615790.5 CTG
PEG10 ENST00000612941.2 CTG

PIGX ENST00000392391.9 CTG

POU5F1 ENST00000606567.6 CTG

PRPS1L1 ENST00000506618.5 ACG

PRR16 ENST00000446965.2 GTG

PTEN ENST00000693560.1 CTG

R3HCC1 ENST00000522012.6 CTG
R3HCC1 ENST00000625275.3 CTG
R3HCC1 ENST00000265806.12 CTG

RNF187 ENST00000305943.9 CTG

RPTOR ENST00000697423.1 CTG

SFPQ ENST00000696553.1 GTG

SLC22A17 ENST00000397260.7 CTG
SLC22A17 ENST00000397267.6 CTG
SLC22A17 ENST00000474057.6 CTG
SLC22A17 ENST00000354772.10 CTG
SLC22A17 ENST00000636431.2 CTG

SLC6A1 ENST00000698198.1 CTG, there is dATG. no ribo-seq. why? 

SORCS1 ENST00000612154.4 ATA   ATA right in front of ATG. there is upstream ATG. no ribo-seq. removed? 
SORCS1 ENST00000622431.4 ATA

SP3 ENST00000418194.7 ATA
SP3 ENST00000652005.2 ATA

STIM2 ENST00000467011.6 TTG
STIM2 ENST00000465503.6 TTG
STIM2 ENST00000467087.7 TTG

SWI5 ENST00000608796.6 CTG
SWI5 ENST00000652598.1 CTG
SWI5 ENST00000419867.7 CTG
SWI5 ENST00000418976.3 CTG
SWI5 ENST00000696406.1 CTG

TEAD1 ENST00000527636.7 ATT
TEAD1 ENST00000527575.6 ATT

TEAD3 ENST00000402886.9 ATA
TEAD3 ENST00000338863.13 ATA
TEAD3 ENST00000639578.3 ATA

TEAD4 ENST00000358409.7 TTG
TEAD4 ENST00000359864.8 TTG
TEAD4 ENST00000443986.3 TTG

TRPC1 ENST00000698238.1 CTG

TRPV6 ENST00000359396.9 ACG

TUG1 ENST00000644773.3 CTG   ??? 

TXNRD3 ENST00000524230.9 CTG
TXNRD3 ENST00000523403.3 CTG

UFSP1 ENST00000672365.3 CTG

USP19 ENST00000698200.1 AAG ???

VANGL2 ENST00000696602.1 ATA

VEGFA ENST00000672860.3 CTG
VEGFA ENST00000372067.8 CTG
VEGFA ENST00000425836.8 CTG

WDR26 ENST00000414423.9 ACG
WDR26 ENST00000678555.1 ACG
WDR26 ENST00000678307.1 ACG

WT1 ENST00000452863.10 CTG
WT1 ENST00000379077.9 CTG
WT1 ENST00000332351.9 CTG
WT1 ENST00000448076.9 CTG